In [6]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
# import tensorflow_hub as hub

In [ ]:
# ================================================ #
#                  Pretrained-Model                #
# ================================================ #

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

model = keras.models.load_model("pretrained/")    # Ensure that you have a saved model in directoy named pre-trained
model.trainable = False             # Freeze the layers

base_input = model.layers[0].input
base_output = model.layers[-2].output   # Getting uptill the second last layer i.e. removing the final layer
final_output = layers.Desne(20)(base_output)

model_new = keras.Model(inputs = base_input, outputs = final_output)

model_new.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

model_new.fit(x_train, y_train, batch_size=32, epochs=3, verbose=2)

In [7]:
# =================================================== #
#                Pretrained Keras Model               #
# =================================================== #

# Random data for demonstration (3 examples w. 3 classes)
x = tf.random.normal(shape=(3, 299, 299, 3))
y = tf.constant([0, 1, 2])

model = keras.applications.InceptionV3(include_top=True)          # Loading InceptionV3 from Keras
model.trainable = False 

base_inputs = model.layers[0].input
base_outputs = model.layers[-2].output
classifier = layers.Dense(3)(base_outputs)
new_model = keras.Model(inputs=base_inputs, outputs=classifier)
new_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

new_model.fit(x, y, epochs=15, verbose=2)

96116736/96112376 [==============================] - 22s 0us/step
Epoch 1/15
1/1 - 31s - loss: 1.2145 - accuracy: 0.3333
Epoch 2/15
1/1 - 0s - loss: 1.0500 - accuracy: 0.6667
Epoch 3/15
1/1 - 0s - loss: 0.9781 - accuracy: 0.6667
Epoch 4/15
1/1 - 0s - loss: 0.9346 - accuracy: 0.3333
Epoch 5/15
1/1 - 0s - loss: 0.8714 - accuracy: 0.3333
Epoch 6/15
1/1 - 0s - loss: 0.7949 - accuracy: 1.0000
Epoch 7/15
1/1 - 0s - loss: 0.7221 - accuracy: 1.0000
Epoch 8/15
1/1 - 0s - loss: 0.6605 - accuracy: 1.0000
Epoch 9/15
1/1 - 0s - loss: 0.6092 - accuracy: 1.0000
Epoch 10/15
1/1 - 0s - loss: 0.5641 - accuracy: 1.0000
Epoch 11/15
1/1 - 0s - loss: 0.5209 - accuracy: 1.0000
Epoch 12/15
1/1 - 0s - loss: 0.4775 - accuracy: 1.0000
Epoch 13/15
1/1 - 0s - loss: 0.4344 - accuracy: 1.0000
Epoch 14/15
1/1 - 0s - loss: 0.3941 - accuracy: 1.0000
Epoch 15/15
1/1 - 0s - loss: 0.3583 - accuracy: 1.0000


In [ ]:
# ================================================= #
#                Pretrained Hub Model               #
# ================================================= #

# Random data for demonstration (3 examples w. 3 classes)
x = tf.random.normal(shape=(3, 299, 299, 3))
y = tf.constant([0, 1, 2])

url = "https://tfhub.dev/google/imagenet/inception_v3/feature_vector/4"

base_model = hub.KerasLayer(url, input_shape=(299, 299, 3))
base_model.trainable = False

model_new = keras.Sequential(
    [
        base_model,
        layers.Dense(128, activation="relu"),
        layers.Dense(64, activation="relu"),
        layers.Dense(10),
    ]
)

model_new.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

model_new.fit(x, y, batch_size=32, epochs=15, verbose=2)